## EfficientNet
- Paper : (2020)_EfficientNet-Rethinking Model Scaling for Convolutional Neural Networks
- 2019 MnasNet 저자이며 Image Classification 타겟의 굉장히 성능이 좋은 Model
- EfficientNet은 depth, width, resolution 을 잘 조합해서 compound network를 만들어 보자는 것.
- ResNet 이후의 Two Streams
    - Better Accuracy (더 높은 성능을 위한)
        - 메모리, 효율성보다는 일단은 성능 위주로 (Large Model)
        - NasNet, MNasNet, ResNet101, SENet, GPipeNet 등.. 
       
    - Best Efficiency
        - 성능은 어느정도 괜찮은 것 같으니.. 효율성을 취하자.
        - MobileNet, ShuffleNet, SqueezeNets 계열 등...
    - etc
        - AutoML (MNasNet) 같은 느낌의 모델 방향도 나옴.

- 해당 논문에서는 MNasNet과 거의 동일한 Search Space하에서 AutoML을 통해 모델을 탐색

<hr>



## 모델의 성능을 올리는 가장 기본적인 접근 3가지 (Model scaling)
- 1. Depth scailing
    - 레이어를 깊게 쌓자
    
- 2. Width scailing
    - 채널수를 늘리는 것
    
- 3. Image Resolution (r) scailing
    - 이미지 해상도를 높이는 것
    
- 3방향에 대해 각각 하는 것이 아니라, 잘 조합된 Compound 된 모델을 뽑아보겠다.

![efficientnet_modelscailing_img](img/efficientnet_modelscailing_img.png)

<hr>


## Problem Formulation
- **실험에 대한 문제를 Formula로 정의**한 것
- Conv NeuralNet(N)이 정의되고, inputtensor(X), spatial dimension (H,W) channel dimension(C)
- F(Base Line)는 정의 및 고정하고, input tensor의 h,w,c를 조정하면서 실험하는 것.
- 하지만 Design space 에 대한 부분은 남아있다.

![efficientnet_formula](img/efficientnet_formula.PNG)

- **Memory나 Flops를 제한(<=target memory, <=target_flops)한채 max(Accuracy(N(d,w,r)) 한 모델을 찾아보겠다**
- d,w,r(depth, width, resolution)

![efficientnet_formula2.PNG](img/efficientnet_formula2.PNG)

<hr>


## Scailing Factor 변동 실험 
- 나머지 Scaling은 고정해두고, 1가지 Scailing Factor에 대해서만 조정해가며 정확도의 변화 측정 
![efficientnet_scaling_each](img/efficientnet_scaling_each.png)

- 1. Scaling Dimensions - Width(Channel) 
    - Channel 수를 늘리게 되면 더 fine-grained features들을 뽑아주고, 학습도 더 잘된다.
    - 하지만 wideResNet처럼 많이 늘리다보면 어느정도 올라가서 Saturated된 모습을 보인다.
    
- 2. Scaling Dimensions - Depth
    - Depth를 늘리게되면 어느정도 성능이 올라가지만, 어느정도 Saturated(포화된) 된 모습을 보인다.
    - ResNet-1000이 ResNet-101과 10배가 넘는 Layer를 가지고 있는데도 비슷한 결과를 가진다.

- 3. Scaling Dimensions - Resolution
    - Resolution은 이미지 크기에 따라 성능은 올라가지만 올리는거 대비 accuracy gain은 감소하는 형태가 나타난다. (잘 안올라간다 느낌)
    
#### Observation 1) 어느정도 이상 올라가면 얻는 gain 이 감소하는 모습을 보인다.


![efficientnet_compound_scaling](img/efficientnet_compound_scaling.PNG)
- depth(d)와 resolution(r)을 고정해두고 width만 조절하며 정확도의 변화를 측정하는 실험을 수행

- 초록색 선 & 노란색 선 
    - depth를 키우는 것 보다는 resolution을 키우는 것이 정확도가 좋음
- 빨간색 선
    - 1가지 or 2가지 scaling factor만 키워주는 것 보다 3가지 scaling factor를 동시에 키워주는 것이 성능이 더 좋음

- Resolution 을 키우면 more layers, more channels 가 증가해주는것이 당연하다의 주장
    - Input image가 커지면 그에 따라서 receptive field도 늘려줘야 하며
    - 더 커진 fine-grained pattern들을 학습하기 위해 더 많은 channel이 필요한 건 reasonable 함
    - 어떻게 이 3가지 요소들을 고려할 것인지에 대한 설명 ==> Compound Scaling
    
#### Obeservation 2) Resolution을 키움에 따라 layers와 Channels의 ratio를 맞춰주는 것은 중요하다


<hr>

## Compound Scaling 실험

- **가장 큰 핵심은 모델(F)를 고정하고 depth(d), width(w), resolution(r) 3가지를 조절하는데, 이때 고정하는 모델(F)를 좋은 모델로 선정하는 것이 가장 중요**


- **어떻게 Compound Scaling을 테스트 해볼 것인가?**

![efficientnet_compound_scaling_method](img/efficientnet_compound_scaling_method.PNG)

- depth(α)는 2의 배수를 따르며, width(β), resolution(γ)는 input, output 의 2가지 측면에 따라 제곱된 값의 비율을 따름

### Scail up MobileNets and ResNets
- MobileNet v1, MobileNet v2, ResNet-50에서 그냥 단순 Scale up (width, resolution, depth) 보다 Compound Scale이 더 좋은 것을 확인

![efficientnet_scailup_mobilenet_resnet](img/efficientnet_scailup_mobilenet_resnet.PNG)

<hr>



## EfficientNet Architecture
- MBConv는 Inverted BottleNeck구조 (Mobilenet v2 구조) / L은 반복횟수 / 

![efficientnet_model_b0](img/efficientnet_model_b0.PNG)

- Step1) 파이(Φ)를 1로 지정 = 네트워크가 2배로 키워지는것 = 어떤 값이 가장 좋은 값인가 (GridSearch를 이용한 탐색) => α=1.2, β=1.1, γ=1.15

- Step2) Optimal 한 αβγ를 찾은 상태에서 파이(Φ)를 키워 해당 모델을 EfficientNet-B1 to B7 까지 만듬


### ImageNet Results for EfficientNet
- ConvNet과 AutoML을 통해 찾은 ConvNet들과 비교를 한 결과
- **비슷한 Accuracy를 가졌던 ImageNet모델들에 대해 EfficientNet을 진행한 것**
- **얼마나 Parameter를 효율적으로 쓰지 못했는지 보여줌**
- **즉, ConvNet들에 비해 비슷한 정확도를 보이면서 parameter수와 FLOPS 수를 굉장히 많이 절약할 수 있음**

![efficientnet_architecture](img/efficientnet_architecture.PNG)

![efficientnet_vs_imagenet_result](img/efficientnet_vs_imagenet_result.PNG)

<hr>

## Inference Time Latency

- 추론 시간 비교  / 파라미터차이가 몇십배가 난다고 해서 속도차이도 몇십배가 나는건 아님

![efficientnet_inference_latency](img/efficientnet_inference_latency.PNG)

<hr>

## Transfer Learning Results for EfficientNets
- **유명  Datasets에 대해 Sota 모델에 대한 Transfer Learning 으로 비교했을 때 총 8개중 5개가 능가하는 결과를 보임** 

- 그래프는 더 왼쪽 위에 있어서 보다 효율적인 것

![efficientnet_transferlearning_result](img/efficientnet_transferlearning_result.PNG)

<hr>

## Efficient Net만 가지고 순수 Scaling 비교
- 혹시 해당 Base 모델들에 대해서만 그러한 결과를 보인 것이 아닌지에 대한 반박

![efficientnet_scaliup](img/efficientnet_scaliup.PNG)

## EfficientNet - Class Activation Map

![efficientnet_activation_map](img/efficientnet_activation_map.PNG)



## EfficientNet Result
- Class Activation Map(CAM)

![efficientnet_result](img/efficientnet_result.png)


## 참고문헌
- https://www.youtube.com/watch?v=Vhz0quyvR7I // PR169- EfficientNet
- https://hoya012.github.io/blog/EfficientNet-review/?fbclid=IwAR1gwSEAF8nqcWPhXzb39SYmJ1kQ6X6rFoZWjzmc05-dBHuBKeltagkUDZI // hoya Efficient Net 정리글